In [1]:
!pip install -q "transformers==4.44.2" sentencepiece accelerate nltk



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.3 MB/s eta 0:00:00


In [2]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
!pip install -q "summac==0.0.4" --no-deps


In [4]:
import types, sys

# 给 transformers.models.deprecated.ernie_m 做一个假的空模块
dummy_module = types.ModuleType("transformers.models.deprecated.ernie_m")
sys.modules["transformers.models.deprecated.ernie_m"] = dummy_module

print("patched ernie_m")


patched ernie_m


Load BART model

In [5]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
import torch, numpy as np

print("numpy:", np.__version__)

MODEL_ID = "mercuryujia/bart-large-multi-news"

# 1. 修改 config，让 early_stopping 不为 None
config = AutoConfig.from_pretrained(MODEL_ID)
config.early_stopping = True

# 2. 慢版 tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)

# 3. 使用修改过的 config 加载模型
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, config=config)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("BART 加载完成 on", device)


numpy: 2.0.2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `num_beams` is set to None - defaulting to 1.
  warnings.warn("`num_beams` is set to None - defaulting to 1.", UserWarning)
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

BART 加载完成 on cpu


Generate 4 candidate abstract

In [6]:
from transformers import GenerationConfig

source_document = """
President Joe Biden met with congressional leaders at the White House on Tuesday
to discuss a new economic relief package. The meeting comes as inflation remains
high and lawmakers are divided over the size of government spending.

Meanwhile, the Federal Reserve signaled it may slow down the pace of interest
rate hikes later this year if price increases continue to cool.

Economists say the negotiations in Washington could have major consequences
for financial markets and for American households.
"""

K = 4

gen_config = GenerationConfig(
    bos_token_id=0,
    decoder_start_token_id=2,
    early_stopping=True,
    eos_token_id=[2],
    forced_bos_token_id=0,
    forced_eos_token_id=2,
    length_penalty=2.0,
    max_length=142,
    min_length=56,
    no_repeat_ngram_size=3,
    num_beams=4,
    pad_token_id=1,
)

inputs = tokenizer(
    source_document,
    max_length=1024,
    truncation=True,
    return_tensors="pt"
).to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        generation_config=gen_config,
        num_return_sequences=K,
    )

candidate_summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

print("=== 生成的候选摘要 ===")
for i, s in enumerate(candidate_summaries):
    print(f"[Cand {i}] {s}\n")


=== 生成的候选摘要 ===
[Cand 0] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.

[Cand 1] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the Hill. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.

[Cand 2] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high, and lawmakers are divided over

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import nltk

device = 0 if torch.cuda.is_available() else -1
print("device:", device)

# 1. 加载一个 NLI 模型，这里用 RoBERTa-MNLI
nli_model_name = "roberta-large-mnli"
nli_classifier = pipeline(
    "text-classification",
    model=nli_model_name,
    tokenizer=nli_model_name,
    device=device,
    truncation=True,
)

def nli_factuality_score(doc: str, summary: str) -> float:
    """
    输入：doc（原文），summary（一个候选摘要）
    输出：这个摘要的“被原文支持程度”分数，越高越好。
    做法：把 summary 切句子，每句跟 doc 做 NLI，取 entail 概率平均。
    """
    # 1. 摘要切句
    sentences = nltk.sent_tokenize(summary)
    if not sentences:
        return 0.0

    entail_probs = []

    for sent in sentences:
        # 2. 用 NLI 分类器
        result = nli_classifier(
            {"text": doc, "text_pair": sent},
            top_k=None,
        )
        # result 是一个 list，里面有 ENTAILMENT / NEUTRAL / CONTRADICTION
        # 找到 ENTAILMENT 的概率
        prob_entail = 0.0
        for r in result:
            if r["label"].upper().startswith("ENTAIL"):
                prob_entail = r["score"]
                break
        entail_probs.append(prob_entail)

    # 3. 所有句子的 entail 概率取平均
    return float(sum(entail_probs) / len(entail_probs))


device: -1


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [12]:
factuality_scores = [
    nli_factuality_score(source_document, s)
    for s in candidate_summaries
]

import numpy as np

best_idx = int(np.argmax(factuality_scores))

print("=== All candidate abstract factuality scores ===")
for i, sc in enumerate(factuality_scores):
    print(f"[Cand {i}] score = {sc:.4f}")

print("\n=== Highest factuality score abstract ===")
print(f"Best index: {best_idx}")
print(candidate_summaries[best_idx])


=== All candidate abstract factuality scores ===
[Cand 0] score = 0.0018
[Cand 1] score = 0.0017
[Cand 2] score = 0.0018
[Cand 3] score = 0.0017

=== Highest factuality score abstract ===
Best index: 2
– Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high, and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.
